## Tutorial on the one-class SVM enhanced by CNN features (using ResNet50) with heatmap visualization

## 1. Configurations

In [1]:
from    typing      import  *
from    pathlib     import  Path

import  numpy   as  np
import  torch
from    sklearn.metrics     import  roc_auc_score, classification_report

from    utils       import  *
from    helper      import  *
from    models      import  OC_SVM

device = torch.device('cuda:2')

In [2]:
# Set the root (Category: hazelnut)
base_path = Path(BASE_PATHS[MVTEC_AD])
category = 'bottle'
root = base_path/category

# Set the directories
train_normal_dir    = root / "train" / "good"
test_normal_dir     = root / "test" / "good"
test_anomaly_dirs   = [dir_subcat for dir_subcat in (root/"test").glob('*') if dir_subcat.name != "good"]

## 2. Load data

In [3]:
# Load the train dataset
print(f"Loading the training dataset")
X_train = load_images(train_normal_dir, device=device)

# Load the test dataset
print(f"Loading the test dataset")
## Normal (0)
X_test  = load_images(test_normal_dir, device=device)
y_test  = np.zeros(len(X_test))
## Anomalous (1)
for anomaly_dir in test_anomaly_dirs:
    X_anom = load_images(anomaly_dir, device=device)
    X_test = torch.vstack([X_test, X_anom])
    y_test = np.hstack([y_test, np.ones(len(X_anom))])

Loading the training dataset


good:   0%|          | 0/209 [00:00<?, ?it/s]

Loading the test dataset


good:   0%|          | 0/20 [00:00<?, ?it/s]

broken_large:   0%|          | 0/20 [00:00<?, ?it/s]

broken_small:   0%|          | 0/22 [00:00<?, ?it/s]

contamination:   0%|          | 0/21 [00:00<?, ?it/s]

In [4]:
extractor = ConvFeatureMap(device)
extractor.eval()
with torch.inference_mode():
    feat_train = extractor.forward(X_train).mean((-2, -1)).cpu()
    feat_test  = extractor.forward(X_test).mean((-2, -1)).cpu()
    H, W = feat_train.shape[-2:]
    feat_train = feat_train.flatten(1).numpy()
    feat_test  = feat_test.flatten(1).numpy()
print("Train feature shape:", feat_train.shape)
print("Test feature shape: ", feat_test.shape)

Train feature shape: (209, 2048)
Test feature shape:  (83, 2048)


## 3. Instantiate and train the model

In [5]:
model = OC_SVM(kernel='rbf', gamma='scale', nu=0.05)
model.fit(feat_train)

## 5. Test the model

In [ ]:
# ROC AUC without threshold
scores = model.decision_function(feat_test)
"""Higher value indicates higher probability of being predicted as an outlier (`1`); lower value indicates higher probability of being predicted as an inlier (`0`)."""
auc = roc_auc_score(y_test, scores)
print(f"ROC-AUC: {auc:.4f}")

# Setting the threshold (Example: 0)
y_pred = model.predict(feat_test)   # 1 for inliers, -1 for outliers
y_pred = np.where(y_pred==-1, 1, 0) # inlier: 1 -> 0 / outlier: -1 -> 1

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))

ROC-AUC: 0.9873

Classification Report:
              precision    recall  f1-score   support

         0.0     0.8182    0.9000    0.8571        20
         1.0     0.9672    0.9365    0.9516        63

    accuracy                         0.9277        83
   macro avg     0.8927    0.9183    0.9044        83
weighted avg     0.9313    0.9277    0.9288        83



End of file